In [ ]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product


# 网络分析与可视化
import networkx as nx
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

In [ ]:
# 读取已经处理好的完整语义网络数据

df_complete = pd.read_csv('01_Processed Data/Complete-Data.csv')
df_complete.head()

In [ ]:
# 搜索出概念对应的所有语义特征及其激活状态，并按照 线索度cue_validity 排序 
def List_Semantic_Feature(df, concept):
    return df[df['Concept']==concept].sort_values(by=['cue_validity'],ascending=False)[['Feature','cue_validity']]


def get_children(df, row_name):
    classes = set()
    for i, row in df.iterrows():
        classes.add(row[row_name])
    return classes

In [ ]:
list(List_Semantic_Feature(df_complete,'牛奶')['Feature'])[0]

## 生成命名测试用数据 Picture Naming Test Data Generation

In [ ]:
## Input 
# word_list = ['豆浆', '菠萝汁', '咖啡']
word_list = cue_words

data = {
    'currentStep': 0,
    'steps': []
}

for word in word_list:
    new_step = {
        'name':word,
        # 'image':'local-resource:///Users/zhanghexin/aphasia_viz/src/assets/test_images/'+word+'.jpeg',
        'image':'/test_images/'+word+'.jpeg',
        'countdown':20,
        'result':'fail',
        'status':'unchecked',
    }
    data['steps'].append(new_step)

with open("test_picture-naming.json", "w") as f: 
    json.dump(data, f, ensure_ascii=False)

In [ ]:
## Input 
# word_list = ['豆浆', '菠萝汁', '咖啡']
word_list = cue_words


data = {
    'currentStep': 0,
    'steps': [],
}

for word in word_list:
    new_step = {
        'name': word,
        # 'image':'local-resource:///Users/zhanghexin/aphasia_viz/src/assets/test_images/'+word+'.jpeg',
        'image':'/test_images/'+word+'.jpeg',

        'countdown':20,
        'result':'fail',
        'status':'unchecked',
        'response':'',
        'tab': [],
    }

    concept = word
    rel_list = list(df_complete['Rel'].unique())
    
    for rel in rel_list:
        feature_type = rel+'-'
        data_tab = {
            'feature_type':feature_type,
            'dataSource':[]
        }
        
        feature_list = list(df_complete[(df_complete['Concept']==concept)&(df_complete['Rel']==rel)].sort_values(by=['cue_validity'],ascending=False)['Feature'].unique())[0:10]
        for feature in feature_list:
            cue_validity = "%.3f" % float(df_complete[(df_complete['Concept']==concept)&(df_complete['Feature']==feature)]['cue_validity'])
            related_concepts = List_Related_Concepts(concept,feature)[0:5]
            confused_feature = List_Confused_Feature(concept,feature)[0:5]
            data_row = {
                # 'key':feature_list.index(feature),
                'key':concept+'/'+feature,
                'name':feature,
                'cue_validity':cue_validity,
                'related_concepts': ', '.join(related_concepts),
                'confused_feature': ', '.join(confused_feature).replace(feature_type,''),
                'status': 'NO',
            }
            data_tab['dataSource'].append(data_row)
        new_step['tab'].append(data_tab)

    data['steps'].append(new_step)

with open("test_SFA.json", "w") as f: 
    json.dump(data, f, ensure_ascii=False)